In [4]:
from sycamore.data import Document
from sycamore import DocSet
from sycamore.llms import OpenAI, OpenAIModels
from sycamore.transforms import COALESCE_WHITESPACE
from sycamore.transforms.partition import SycamorePartitioner
from sycamore.transforms.summarize_images import SummarizeImages
from sycamore.transforms.merge_elements import GreedySectionMerger
from sycamore.utils.time_trace import timetrace
from sycamore.utils.cache import Cache
import sycamore

import pickle
import json
import sys
import pandas as pd
import csv
import os

ctx = sycamore.init()
llm = OpenAI(OpenAIModels.GPT_4O.value)

'''
## USE DOCKER COMMAND BELOW TO RUN NEO4j ##
docker run -d --rm \
   -p 7474:7474 -p 7687:7687 \
   --name neo4j-apoc \
   -e NEO4J_apoc_export_file_enabled=true \
   -e NEO4J_apoc_import_file_enabled=true \
   -e NEO4J_apoc_import_file_use__neo4j__config=true \
   -e NEO4J_PLUGINS=\[\"apoc\"\] \
   -e NEO4J_AUTH=neo4j/koala-stereo-comedy-spray-figure-6974 \
   -v ~/neo4j/import:/import \
   neo4j:5.21.0

sudo chmod -R 755 /home/admin/neo4j/import

docker exec -it neo4j-apoc bin/neo4j-admin database import full --nodes=import/nodes.csv --trim-strings=true
bin/neo4j stop
bin/neo4j-admin database import full --nodes=import/nodes.csv --trim-strings=true
   -v ~/sycamore/data/import:/var/lib/neo4j/import \

'''


'\n## USE DOCKER COMMAND BELOW TO RUN NEO4j ##\ndocker run -d --rm    -p 7474:7474 -p 7687:7687    --name neo4j-apoc    -e NEO4J_apoc_export_file_enabled=true    -e NEO4J_apoc_import_file_enabled=true    -e NEO4J_apoc_import_file_use__neo4j__config=true    -e NEO4J_PLUGINS=\\["apoc"\\]    -e NEO4J_AUTH=neo4j/koala-stereo-comedy-spray-figure-6974    -v ~/neo4j/import:/import    neo4j:5.21.0\n\nsudo chmod -R 755 /home/admin/neo4j/import\n\ndocker exec -it neo4j-apoc bin/neo4j-admin database import full --nodes=import/nodes.csv --trim-strings=true\nbin/neo4j stop\nbin/neo4j-admin database import full --nodes=import/nodes.csv --trim-strings=true\n   -v ~/sycamore/data/import:/var/lib/neo4j/import \n'

In [19]:
def elements_to_section(doc: Document) -> Document:
    import uuid
    
    #if the first element is not a section header, insert generic placeholder
    if(len(doc.elements) > 0 and doc.elements[0]["type"] != 'Section-header'):
        initial_page = {'type': 'Section-header', 'bbox': (0, 0, 0, 0), 'properties': {'score': 1, 'page_number': 1}, 'text_representation': 'Front Page', 'binary_representation': b'Front Page'}
        doc.elements.insert(0,initial_page) # O(n) insert :( we should use deque for everything

    if 'relationships' not in doc.data:
        doc.data['relationships'] = {}
    if 'node_props' not in doc.data:
        doc.data['node_props'] = {'label': 'DOCUMENT'}

    section_id = None
    element_id = None
    for e in doc.elements:
        if 'doc_id' not in e:
            e['doc_id'] = str(uuid.uuid4())
        if 'relationships' not in e:
            e['relationships'] = {}
        if 'node_props' not in e:
            e['node_props'] = {}
        if e["type"] == 'Section-header' and len(e["text_representation"]) > 0:
            if section_id != None:
                next = {'START_ID': section_id,'END_ID':e['doc_id'],'TYPE':'NEXT', 'properties': {}}
                e['relationships'][str(uuid.uuid4())] = next
                element_id = None
            relation = {'START_ID': e['doc_id'],'END_ID':doc.doc_id,'TYPE':'SECTION_OF', 'properties': {}}
            e['relationships'][str(uuid.uuid4())] = relation
            e['node_props']['label'] = 'SECTION'
            section_id = e['doc_id']
        else:
            if element_id != None:
                next = {'START_ID': element_id,'END_ID':e['doc_id'],'TYPE':'NEXT', 'properties': {}}
                e['relationships'][str(uuid.uuid4())] = next
            relation = {'START_ID': e['doc_id'],'END_ID':section_id, 'TYPE':'PART_OF', 'properties': {}}
            e['relationships'][str(uuid.uuid4())] = relation
            e['node_props']['label'] = 'ELEMENT'
            element_id = e['doc_id']
    return doc

In [13]:
s3_path = "s3://aryn-datasets-us-east-1/financebench-large/3M_1/"
docset = (
    ctx.read.binary(s3_path, binary_format="pdf")
    .partition(partitioner=SycamorePartitioner(extract_table_structure=True, use_ocr=True, extract_images=True), num_gpus=0.1)
    .regex_replace(COALESCE_WHITESPACE)
    .map(elements_to_section)
    .explode()
)
docs = docset.take_all()

pickle.dump(docs, open('extracted_1.pickle', 'wb'))

ERROR:root:Unable to load aryn config /home/admin/.aryn/config.yaml: [Errno 2] No such file or directory: '/home/admin/.aryn/config.yaml'
2024-07-12 17:18:52,159	WARNING util.py:560 -- The argument ``compute`` is deprecated in Ray 2.9. Please specify argument ``concurrency`` instead. For more information, see https://docs.ray.io/en/master/data/transforming-data.html#stateful-transforms.
2024-07-12 17:18:52,166	INFO streaming_executor.py:112 -- Starting execution of Dataset. Full logs are in /tmp/ray/session_2024-07-12_17-17-15_333127_294963/logs/ray-data
2024-07-12 17:18:52,166	INFO streaming_executor.py:113 -- Execution plan of Dataset: InputDataBuffer[Input] -> TaskPoolMapOperator[ReadBinary] -> TaskPoolMapOperator[Map(BinaryScan._to_document)] -> ActorPoolMapOperator[MapBatches(BaseMapTransformCallable___wrap)] -> TaskPoolMapOperator[MapBatches(regex_replace)->MapBatches(elements_to_section)->MapBatches(explode)]


- ReadBinary->SplitBlocks(16) 1:   0%|          | 0/1 [00:00<?, ?it/s]

- Map(BinaryScan._to_document) 2:   0%|          | 0/1 [00:00<?, ?it/s]

- MapBatches(BaseMapTransformCallable___wrap) 3:   0%|          | 0/1 [00:00<?, ?it/s]

- MapBatches(regex_replace)->MapBatches(elements_to_section)->MapBatches(explode) 4:   0%|          | 0/1 [00:…

Running 0:   0%|          | 0/1 [00:00<?, ?it/s]

(MapWorker(MapBatches(BaseMapTransformCallable___wrap)) pid=297020) The `max_size` parameter is deprecated and will be removed in v4.26. Please specify in `size['longest_edge'] instead`.
(MapWorker(MapBatches(BaseMapTransformCallable___wrap)) pid=297020) /home/admin/.cache/pypoetry/virtualenvs/sycamore-monorepo-RdRTS9RD-py3.11/lib/python3.11/site-packages/torch/utils/cpp_extension.py:1967: UserWarning: TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
(MapWorker(MapBatches(BaseMapTransformCallable___wrap)) pid=297020) If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'].
(MapWorker(MapBatches(BaseMapTransformCallable___wrap)) pid=297020)   warnings.warn(


In [5]:
docs = pickle.load(open('extracted_1.pickle', 'rb'))

In [9]:
print(docs[0])

{'properties': {'path': 's3://aryn-datasets-us-east-1/financebench-large/3M_1/3M_2015_10K.pdf', 'filetype': 'application/pdf'}, 'elements': [], 'lineage_id': 'b51227e5-2b9f-4793-8c70-ef6676745fe3', 'doc_id': 'd29dccd8-4072-11ef-a153-0e5677db27ff', 'type': 'pdf', 'binary_representation': b'%PDF-1.4\n1 0 obj\n<<\n/Title (\xfe\xff\x001\x000\x00-\x00K\x00 \x00-\x00 \x000\x002\x00/\x001\x001\x00/\x002\x000\x001\x006\x00 \x00-\x00 \x003\x00M\x00 \x00C\x00o\x00m\x00p\x00a\x00n\x00y)\n/Creator (\xfe\xff\x00w\x00k\x00h\x00t\x00m\x00l\x00t\x00o\x00p\x00d\x00f\x00 \x000\x00.\x001\x002\x00.\x005)\n/Producer (\xfe\xff\x00Q\x00t\x00 \x004\x00.\x008\x00.\x007)\n/CreationDate (D:20230105154621-05\'00\')\n>>\nendobj\n3 0 obj\n<<\n/Type /ExtGState\n/SA true\n/SM 0.02\n/ca 1.0\n/CA 1.0\n/AIS false\n/SMask /None>>\nendobj\n4 0 obj\n[/Pattern /DeviceRGB]\nendobj\n5 0 obj\n<<\n/Type /Page\n/Parent 2 0 R\n/Contents 10 0 R\n/Resources 12 0 R\n/Annots 13 0 R\n/MediaBox [0 0 612 792]\n>>\nendobj\n12 0 obj\n<<\n

In [21]:
# s3_path = "s3://aryn-datasets-us-east-1/ntsb-large/100/193893.pdf"
# docset = (
#     ctx.read.binary(s3_path, binary_format="pdf")
#     .partition(partitioner=SycamorePartitioner(extract_table_structure=True, use_ocr=True, extract_images=True), num_gpus=0.1)
#     .regex_replace(COALESCE_WHITESPACE)
#     .map(elements_to_section)
#     .explode()
# )
# docs = docset.take_all()


ERROR:root:Unable to load aryn config /home/admin/.aryn/config.yaml: [Errno 2] No such file or directory: '/home/admin/.aryn/config.yaml'
2024-07-12 18:20:28,321	WARNING util.py:560 -- The argument ``compute`` is deprecated in Ray 2.9. Please specify argument ``concurrency`` instead. For more information, see https://docs.ray.io/en/master/data/transforming-data.html#stateful-transforms.
2024-07-12 18:20:28,328	INFO streaming_executor.py:112 -- Starting execution of Dataset. Full logs are in /tmp/ray/session_2024-07-12_18-01-05_180845_304452/logs/ray-data
2024-07-12 18:20:28,328	INFO streaming_executor.py:113 -- Execution plan of Dataset: InputDataBuffer[Input] -> TaskPoolMapOperator[ReadBinary] -> TaskPoolMapOperator[Map(BinaryScan._to_document)] -> ActorPoolMapOperator[MapBatches(BaseMapTransformCallable___wrap)] -> TaskPoolMapOperator[MapBatches(regex_replace)->MapBatches(elements_to_section)->MapBatches(explode)]


- ReadBinary->SplitBlocks(16) 1:   0%|          | 0/1 [00:00<?, ?it/s]

- Map(BinaryScan._to_document) 2:   0%|          | 0/1 [00:00<?, ?it/s]

- MapBatches(BaseMapTransformCallable___wrap) 3:   0%|          | 0/1 [00:00<?, ?it/s]

- MapBatches(regex_replace)->MapBatches(elements_to_section)->MapBatches(explode) 4:   0%|          | 0/1 [00:…

Running 0:   0%|          | 0/1 [00:00<?, ?it/s]

(MapWorker(MapBatches(BaseMapTransformCallable___wrap)) pid=312300) The `max_size` parameter is deprecated and will be removed in v4.26. Please specify in `size['longest_edge'] instead`.
(MapWorker(MapBatches(BaseMapTransformCallable___wrap)) pid=312300) /home/admin/.cache/pypoetry/virtualenvs/sycamore-monorepo-RdRTS9RD-py3.11/lib/python3.11/site-packages/torch/utils/cpp_extension.py:1967: UserWarning: TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
(MapWorker(MapBatches(BaseMapTransformCallable___wrap)) pid=312300) If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'].
(MapWorker(MapBatches(BaseMapTransformCallable___wrap)) pid=312300)   warnings.warn(


In [22]:
include = ['type','bbox', 'text_representation']

def to_csv(docs: DocSet):
    nodes = []
    relationships = []
    for doc in docs:
        node = {
            'uuid:ID' : doc.doc_id,
            ':LABEL' : doc.data['node_props']['label']
        }
        if(doc['type'] == 'table'):
            doc['text_representation'] = doc['table'].to_csv()
        for key,value in doc.data.items():
            #add flatten properties to node
            if(key == 'properties'):
                properties_flat = pd.json_normalize(value, sep='_').to_dict(orient='records')[0]
                for key, value in properties_flat.items():
                    if key not in include:
                        node[key] = value
            #add included fields to node
            if(key in include):
                node[key] = value
        nodes.append(node)
        for key, value in doc.data['relationships'].items():
            rel = {
            'uuid:ID' : key,
            ':START_ID': value['START_ID'],
            ':END_ID': value['END_ID'],
            ':TYPE': value['TYPE']
            }
            for key, value in value['properties'].items():
                if key not in ['uuid:ID',':START_ID',':END_ID',':TYPE']:
                    rel[key] = value

            relationships.append(rel)
        
    return [nodes,relationships]


In [23]:
nodes, relationships = to_csv(docs)

In [25]:
relationships

[{'uuid:ID': '99c073c7-9402-4197-9345-07a6fe35ab53',
  ':START_ID': '8a76c16d-3da4-4912-92da-13f6c683d843',
  ':END_ID': '6df14252-407b-11ef-a153-0e5677db27ff',
  ':TYPE': 'SECTION_OF'},
 {'uuid:ID': '3f5ea912-91a4-4c9d-b388-7eefd43195aa',
  ':START_ID': '25f70c29-75d1-4644-a1ec-8ed9500dfb0c',
  ':END_ID': '8a76c16d-3da4-4912-92da-13f6c683d843',
  ':TYPE': 'PART_OF'},
 {'uuid:ID': 'b85456e8-c096-4a8c-98ae-f858f78d0063',
  ':START_ID': '8a76c16d-3da4-4912-92da-13f6c683d843',
  ':END_ID': 'd3c6ed7d-f723-4045-8215-46bd37e8847b',
  ':TYPE': 'NEXT'},
 {'uuid:ID': '0a5ce14d-d5d8-4994-8d14-0cfceae7d9d0',
  ':START_ID': 'd3c6ed7d-f723-4045-8215-46bd37e8847b',
  ':END_ID': '6df14252-407b-11ef-a153-0e5677db27ff',
  ':TYPE': 'SECTION_OF'},
 {'uuid:ID': '31891a03-b25a-472f-9369-54e511ed7ee0',
  ':START_ID': 'ddb35668-bbac-4ede-856e-c06d617b6a69',
  ':END_ID': 'd3c6ed7d-f723-4045-8215-46bd37e8847b',
  ':TYPE': 'PART_OF'},
 {'uuid:ID': 'b9ced4c1-7e0b-48af-ae94-586412ec7ea7',
  ':START_ID': 'ddb35668

In [26]:
headers = sorted(set(key for entry in nodes for key in entry.keys()))
csv_path = os.path.expanduser('~/neo4j/import/nodes.csv')
os.makedirs(os.path.dirname(csv_path), exist_ok=True)
with open(csv_path, 'w', newline='') as file:
    writer = csv.writer(file)
    
    writer.writerow(headers)
    
    for entry in nodes:
        row = [entry.get(header, '') for header in headers]
        writer.writerow(row)

headers = sorted(set(key for entry in relationships for key in entry.keys()))
csv_path = os.path.expanduser('~/neo4j/import/relationships.csv')
os.makedirs(os.path.dirname(csv_path), exist_ok=True)
with open(csv_path, 'w', newline='') as file:
    writer = csv.writer(file)
    
    writer.writerow(headers)
    
    for entry in relationships:
        row = [entry.get(header, '') for header in headers]
        writer.writerow(row)

In [27]:
build_nodes = '''
LOAD CSV WITH HEADERS FROM "file:///nodes.csv" AS row

// Create a node with dynamic label and uuid property
CALL apoc.create.node([row[":LABEL"]], {uuid: row["uuid:ID"]}) YIELD node

// Iterate through the row's keys to add properties to the node
WITH row, node
UNWIND keys(row) AS key

// Exclude special keys used for label and uuid
WITH row, node, key
WHERE key <> ":LABEL" AND key <> "uuid:ID"

// Only add non-null properties
CALL apoc.create.setProperty(node, key, row[key]) YIELD node AS n

RETURN n
'''

build_relationships = '''
LOAD CSV WITH HEADERS FROM "file:///relationships.csv" AS row

// Find the start and end nodes using the UUIDs
MATCH (startNode {uuid: row[":START_ID"]}), (endNode {uuid: row[":END_ID"]})

// Create the relationship with dynamic type and an empty properties map
CALL apoc.create.relationship(startNode, row[":TYPE"], {uuid: row["uuid:ID"]}, endNode) YIELD rel

// Iterate through the row's keys to add properties to the relationship
WITH row, rel
UNWIND keys(row) AS key

// Exclude special keys used for start ID, end ID, and type
WITH row, rel, key
WHERE key <> ":START_ID" AND key <> ":END_ID" AND key <> ":TYPE" AND key <> "uuid:ID"

// Only add non-null properties
CALL apoc.create.setRelProperty(rel, key, row[key]) YIELD rel as updatedRel

RETURN updatedRel
'''

In [28]:
from neo4j import GraphDatabase
# Define a Neo4jHandler class to handle Neo4j operations
class Neo4jHandler:
    def __init__(self, uri, auth):
        self.driver = GraphDatabase.driver(uri, auth=auth)
        self.session = self.driver.session(database="neo4j")

    def close(self):
        self.session.close()
        self.driver.close()

    def wipe_all_data(self):
        with self.session.begin_transaction() as tx:
            tx.run("MATCH (n) DETACH DELETE n")

    def load_csv(self, command):
        with self.session.begin_transaction() as tx:
            tx.run(command)

URI = "neo4j://localhost:7687"
AUTH = ("neo4j", "koala-stereo-comedy-spray-figure-6974")
neo4j_handler = Neo4jHandler(URI, AUTH)


In [29]:
neo4j_handler.wipe_all_data()
neo4j_handler.load_csv(build_nodes)
neo4j_handler.load_csv(build_relationships)

In [ ]:
# from sycamore.utils.pdf_utils import show_pages
# sample_pages = show_pages(docset, limit=20)

In [35]:
import json
import csv
import io

# get the JSON objects from JSONL
jsonl_data = """

"""
json_lines = tuple(json_line
                   for json_line in jsonl_data.splitlines()
                   if json_line.strip())
jsons_objs = tuple(json.loads(json_line)
                   for json_line in json_lines)


headers = sorted(set(key for entry in jsons_objs for key in entry.keys()))

csv_path = os.path.expanduser('~/sycamore/financebench/relationships.csv')
os.makedirs(os.path.dirname(csv_path), exist_ok=True)
with open(csv_path, 'w', newline='') as file:
    writer = csv.writer(file)
    
    writer.writerow(headers)
    
    writer.writerows((value for key, value in sorted(json_obj.items()))
                 for json_obj in jsons_objs)

#

#print(fake_file.getvalue())

JSONDecodeError: Unterminated string starting at: line 1 column 647 (char 646)